We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.


Here are a few of the high-level components we'll be working with:

- **Chat Models.** The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat Models rather than text LLMs.
- **Prompt Templates,** which simplify the process of assembling prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
- **Chat History,** which allows a chatbot to "remember" past interactions and take them into account when responding to followup questions.
- **Debugging and tracing** your application using LangSmith
We'll cover how to fit the above components together to create a powerful conversational chatbot.

# Setup

In [ ]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Langchain-chatbot"
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')

In [ ]:
! pip install -q langchain
! pip install -q cohere
! pip install -q langchain-community


# model selection

In [ ]:
! pip install -qU langchain-cohere

In [ ]:
from langchain_cohere import ChatCohere
model = ChatCohere(model="command-r")

# Chat Model

ChatModels are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the .invoke method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model.invoke([HumanMessage(content="Hi! My name is Ram.")])

AIMessage(content="Hi Ram! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '82df93df-ed4c-41da-b9b9-374eb7adbc3c', 'token_count': {'input_tokens': 73, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '82df93df-ed4c-41da-b9b9-374eb7adbc3c', 'token_count': {'input_tokens': 73, 'output_tokens': 16}}, id='run-9f65dc67-144f-43b4-8d1f-a80cabfcf994-0')

The model on its own does not have any concept of state. For example, if you ask a followup question:



In [ ]:
model.invoke([HumanMessage("What is my name?")])

AIMessage(content="I'm sorry, but as an AI chatbot, I don't know your name. The name is something that you did not share with me, and I cannot guess it either. However, if you tell me your name, I'll be happy to address you by it.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9e8baad1-1efe-4efa-9204-174f0b90fa17', 'token_count': {'input_tokens': 71, 'output_tokens': 55}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9e8baad1-1efe-4efa-9204-174f0b90fa17', 'token_count': {'input_tokens': 71, 'output_tokens': 55}}, id='run-097503b8-d1e4-4882-a842-b256da966290-0')

 To make the model to remember, we need to pass the entire conversation history into the model

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content ="Hi! My name is Ram."),
        AIMessage(content="Hello Ram! It's nice to meet you"),
        HumanMessage(content="What is my name?")
    ]
)

AIMessage(content='Your name is Ram! You mentioned it at the beginning of our conversation. Nice to meet you, Ram!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7b370ccd-7ecc-45b2-81fd-506c46704fb9', 'token_count': {'input_tokens': 93, 'output_tokens': 22}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7b370ccd-7ecc-45b2-81fd-506c46704fb9', 'token_count': {'input_tokens': 93, 'output_tokens': 22}}, id='run-3b6b1253-be63-44b2-aafc-47969c225a46-0')

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally. So how do we best implement this?



# Message History

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_histroy = RunnableWithMessageHistory(runnable=model,get_session_history=get_session_history)

In [ ]:
config = {"configurable": {"session_id": "123"}}

In [ ]:
response = with_message_histroy.invoke(
    [HumanMessage(content="Hi! My name is Ram.")],
    config=config,
)
response.content


"Hi Ram! It's nice to meet you. How's it going today?"

In [ ]:
response = with_message_histroy.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)
response.content

'Your name is Ram! You mentioned it at the beginning of our conversation.'

Great! Our chatbot now remembers things about us. If we change the config to reference a different session_id, we can see that it starts the conversation fresh.



In [ ]:
config = {"configurable": {"session_id": "456"}}
response = with_message_histroy.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)
response.content

"I'm sorry, but as an AI chatbot, I don't know your name. The name is something that you did not share with me, and I cannot guess it either. However, if you tell me your name, I'll be happy to address you by it."

In [ ]:
config = {"configurable": {"session_id": "123"}}
response = with_message_histroy.invoke(
    [HumanMessage(content="Hi! My name is Ram.")],
    config=config,
)
response.content

"Hello Ram! It's great to hear from you. Are you named after anyone or is it a name you really like?"

# Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpfull assistant"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [ ]:
chain = prompt | model

In [ ]:
response = chain.invoke(
    {
        "messages":[
            HumanMessage(content="Hi! My name is Ram.")
            ]
    }
)
response.content

"Hi Ram! It's nice to meet you. How's your day been so far?"

We can now wrap this in the same Messages History object as before



In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history
)

config = {"configurable": {"session_id": "ram11"}}
response = with_message_histroy.invoke(
    [HumanMessage(content="Hi! My name is Ram.")],
    config=config,
)
response.content



"Hello Ram! It's lovely to meet you. I hope you're doing well and having a wonderful day. Do you want to talk about anything specific, or just have a general chat?"

In [ ]:
response = with_message_histroy.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)
response.content


"Your name is Ram! It's a pleasure to have you here, Ram. Is there anything you would like to talk about?"

Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {
        "messages":[HumanMessage(content="Hi! My name is Ram.")],
        "language":"Spanish"
    }
)
response.content

'¡Hola, Ram! Me alegra conocerte.'

In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [ ]:
config = {"configurable": {"session_id": "abc11"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content="Hi! My name is Ram.")],
        "language": "Spanish"
    },
    config=config,
)
response.content


'¡Hola, Ram! Me alegra conocerte.'

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Ram. ¡Un nombre muy interesante!'

# Managing Conversation History

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(
        messages = lambda x: filter_messages(x["messages"])
    )
    | prompt
    | model
)

In [ ]:
messages = [
    HumanMessage(content="hi! I'm Ram"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I can't help you with that. You haven't introduced yourself yet! Who would you like your name to be?"

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'You seem to like vanilla ice cream! But to confirm, what is your favorite ice cream flavor?'

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I can't help you with that as you haven't mentioned your name yet. If you'd like, I could help you come up with some name suggestions. I've helped other users with this in the past and it's quite fun! There are also some cool baby name websites we could use. Would you like me to give you some name suggestions?"

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

"Since I don't know who you are, I don't know your favorite ice cream flavor. However, some popular ice cream flavors include chocolate, vanilla, and strawberry. Do you want to know anything else about ice cream?"

# Streaming

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Hello| Todd|!| This| one|'s| for| you|:|

What| do| you| call| a| factory| that| makes| distinctly| average| products|?| 

An| adequate| factory|!|

Have| a| great| day|,| Todd|!|Hello Todd! This one's for you:

What do you call a factory that makes distinctly average products? 

An adequate factory!

Have a great day, Todd!|